In [1]:
# initial imports 
import pandas as pd
from gekko import GEKKO
import numpy as np 
import seaborn as sns 
import logging
from functools import lru_cache
import ast 
import os 
import pandas as pd
from datetime import date

import matplotlib.pyplot as plt

In [2]:
# HCP DIGI 
hcp_digi_m1 = 6.01286785e+02
hcp_digi_m2 = -3.39198693e-01 
hcp_digi_m3 = 6.35363532e-05
hcp_digi_int_C = 67555.27887	

# HCP MAIL 
hcp_mail_m1 = 2.97917081e+03
hcp_mail_m2 = -1.71565008e+00 
hcp_mail_int_c = -48721.27889

# DTC DISPLAY 

dtc_disp_m1 = 3.45276779e+04
dtc_disp_m2 =  -1.65277192e+01  
dtc_disp_m3 = 5.93452413e-03
dtc_disp_m4 =  -9.42046720e-07
dtc_disp_intc = 192670.1652	

# HCP  DISPLAY 
hcp_disp_m1 =  4.11445774e-01 
hcp_disp_m2 =  1.27940560e+02
hcp_disp_m3 = -2.22402089e-01
hcp_disp_m4 =  1.71248457e-04
hcp_disp_m5 = -6.33547691e-08
hcp_disp_m6 = 9.13303494e-12
hcp_disp_intc = 817530.438

# -- field calls 

# model coeff_
f_m1 = 1.10785653e-03  
f_m2 = 3.42403093e+00 
f_m3 = -4.55404565e-04  
f_m4 = 2.57919190e-08 
f_m5 = -5.53661654e-13
f_Intercept_ = 239297
f_mROI_latch = 1.005172531

# hcp paid search ( corfficients & intercept )
hcp_m1 = 2.83578835e+04  
hcp_m2 = -1.67861082e+01 
hcp_int_c = -572151.0421

#DTC PAID SEARCH ( corfficients & intercept )
dtc_p_search_m1 = 1.71479676e+03 
dtc_p_search_m2 = -1.75357980e-01
dtc_p_search_int_c = -245074.9513

# Speaker program Person 

sp_p_m1 = 4.76886710e+01  
sp_p_m2 =  -3.09063497e-02 
sp_p_m3 =  6.78852771e-06
sp_p_intc = -744.7706042

# Speaker program Virtual 
sp_v_m1 = 4.21116616e+02 
sp_v_m2 = -2.80086720e-01 
sp_v_m3 = 5.98234666e-05
sp_v_intc = 45819.06696



# GNA CALLS 
gna_call_m1 = 1.53789502e+04
gna_call_m2 =  -1.56081127e+01  
gna_call_m3 = 7.02161425e-03 
gna_call_m4 = -1.17091134e-06
gna_call_intc = -996495.5305

# DTC SOCIAL 
dtc_social_m1 = 1.34750048e+04  
dtc_social_m2 = 9.07146577e+00 
dtc_social_m3 = -1.35022365e-02 
dtc_social_m4 =  5.48239513e-06 
dtc_social_m5 = -7.59241220e-10

dtc_social_intc = -1419139.752

In [91]:
data = pd.read_excel('../Excel_data_input/Reports.xlsx' , sheet_name='current_val')
pd.options.display.float_format = "{:.2f}".format
data.head(11)

,Channel Names,Current Spend,Saturation Point
0,field call,52891109.13,30147932.21
1,dtc paid search,6100000.38,5246000.00
2,hcp paid search,195203.29,199107.00
3,hcp_digital_spend,1468985.00,396626.00
4,hcp_mail_spend,130745.01,160816.00
5,dtc_display_spend,1440000.05,1843200.00
6,hcp_display_spend,1036398.00,1129674.00
7,spk_person_spend,1008083.93,20161.68
8,spk_virtual_spend,1260916.07,12609.16
9,gna_call_spend,1100000.00,616000.00


In [92]:
data['Current Spend'].sum()


68167440.85520014

## -ve analysis ( with ub = Saturation Points)

-  **Upper boundary** = **saturation point** 
-  **Lower Boundary** = **Spend at -99%**           
- **budget : -10 % to -50% of Current spend**

In [33]:
data

,Channel Names,Current Spend,Saturation Point
0,field call,52891109.13,30147932.21
1,dtc paid search,6100000.38,5246000.00
2,hcp paid search,195203.29,199107.00
3,hcp_digital_spend,1468985.00,396626.00
4,hcp_mail_spend,130745.01,160816.00
5,dtc_display_spend,1440000.05,1843200.00
6,hcp_display_spend,1036398.00,1129674.00
7,spk_person_spend,1008083.93,20161.68
8,spk_virtual_spend,1260916.07,12609.16
9,gna_call_spend,1100000.00,616000.00


In [8]:
data['Current Spend'].sum() - data['Current Spend'].sum()*0.1

61350696.76968013

In [127]:
Total_budget = data['Current Spend'].sum()
budget_10pcnt = data['Current Spend'].sum() - ( data['Current Spend'].sum() * 0.1 )  # -10%
budget_20pcnt = data['Current Spend'].sum() - ( data['Current Spend'].sum() * 0.2 )
budget_30pcnt = data['Current Spend'].sum() - ( data['Current Spend'].sum() * 0.3 )
budget_40pcnt = data['Current Spend'].sum() - ( data['Current Spend'].sum() * 0.4 )
budget_50pcnt = data['Current Spend'].sum() - ( data['Current Spend'].sum() * 0.5 )
budget_60pcnt = data['Current Spend'].sum() - ( data['Current Spend'].sum() * 0.6 )
print('Total Budget  : ' , Total_budget)
print('-10 % ' , budget_10pcnt)
print('-20 % ' ,budget_20pcnt)
print('-30 % ' ,budget_30pcnt )
print('-40 % ' ,budget_40pcnt)
print('-50 % ' ,budget_50pcnt)
# print('-10 % ' ,budget_60pcnt)


Total Budget  :  68167440.85520014
-10 %  61350696.76968013
-20 %  54533952.68416011
-30 %  47717208.5986401
-40 %  40900464.513120085
-50 %  34083720.42760007


In [4]:
total_budget =68167440.85520014

def gekko_ (total_budget) : 
    model_2 = GEKKO()

    total_spend = model_2.Const(total_budget)

    # Constant spend of Websites 
    constant_values = {'hcp_website_spend_' : 483486 ,'dtc_website_spend_' : 400000 , 'hcp_website_revenue_' : 6800000 ,'dtc_website_revenue_' :  9600000 } 

    ##  revenue of websites
    fild_spend = model_2.Var(value=51251485+(-0.10*51251485) , lb=51251485+(-0.10*51251485)  , ub = 51251485+(0.10*51251485), name='Field call spend' )   
    dp_spend = model_2.Var(value= 6100000+(-0.10*6100000) , lb= 6100000+(-0.10*6100000), ub = 6100000+(0.10*6100000) , name='dtc paid search')     # change in upper boundary
    hcp_spend = model_2.Var(value=  238929+(-0.10*238929), lb= 238929+(-0.10*238929) , ub= 238929+(0.10*238929), name='hcp paid search' )   # hcp paid search  
    hcp_digital_spend = model_2.Var(value=  1468985+(-0.10*1468985), lb= 1468985+(-0.10*1468985)  , ub=1468985+(0.10*1468985) , name='hcp digital spend' ) 
    hcp_mail_spend = model_2.Var(value=  192980+(-0.10*192980)  , lb= 192980+(-0.10*192980)  , ub=  192980+(0.10*192980) , name='hcp mail spend' )
    dtc_display_spend = model_2.Var(value=  2027520+(-0.10*2027520), lb= 2027520+(-0.10*2027520)   , ub= 2027520+(0.10*2027520)  , name='dtc display spend'  ) 
    hcp_display_spend = model_2.Var(value=  1242641+(-0.10*1242641)  , lb=  1242641+(-0.10*1242641)   , ub=1242641+(0.10*1242641), name='hcp display spend' ) 
    spk_person_spend = model_2.Var(value=   1008084+(-0.10*1008084) , lb= 1008084+(-0.10*1008084)   , ub= 1008084+(0.10*1008084) , name='spk person spend') 
    spk_virtual_spend = model_2.Var(value=  1260916+(-0.10*1260916), lb=1260916+(-0.10*1260916), ub= 1260916+(0.10*1260916), name='spk virtual spend')     # changes in lower boundary 
    gna_Calls_spend = model_2.Var(value=  1100000+(-0.10*1100000), lb= 1100000+(-0.10*1100000) , ub= 1100000+(0.10*1100000), name='gna call spend')  
    dtc_social_spend = model_2.Var(value= 1993728+(-0.10*1993728), lb= 1993728+(-0.10*1993728) , ub=  1993728+(0.10*1993728) , name='dtc_social_spend')  



    # sqrts  : Sqrt of channel as we have use SQRT + poly transformation 
    Field_sqrt = fild_spend**0.5       
    dp_sqrt = dp_spend**0.5
    hcp_sqrt = hcp_spend**0.5   # hcp paid search 
    hcp_digital_sqrt = hcp_digital_spend**0.5
    hcp_mail_Sqrt = hcp_mail_spend**0.5 
    dtc_display_sqrt = dtc_display_spend**0.5
    hcp_display_sqrt = hcp_display_spend**0.5 
    spk_person_sqrt = spk_person_spend**0.5 
    spk_virtual_Sqrt = spk_virtual_spend**0.5
    gna_call_sqrt = gna_Calls_spend**0.5 
    dtc_social_sqrt  = dtc_social_spend**0.5 

    # Equations 

    y_fild_calls = (Field_sqrt*f_m1) + (Field_sqrt**2)*(f_m2) + (Field_sqrt**3)*(f_m3) + (Field_sqrt**4)*(f_m4) + (Field_sqrt**5)*(f_m5) + f_Intercept_ # Revenue Field calls

    # DTC PAID SEARCH 
    y_dp_search = dp_sqrt*(dtc_p_search_m1) + ((dp_sqrt**2)*(dtc_p_search_m2))  + dtc_p_search_int_c    # Revenue  DTC paid search

    y_hcp_search = hcp_sqrt*(hcp_m1) + ( (hcp_sqrt**2)* hcp_m2) + hcp_int_c # Revenue HCP search 

    # hcp digital 
    y_hcp_digital  = hcp_digital_sqrt*(hcp_digi_m1) +  (hcp_digital_sqrt**2)*hcp_digi_m2 + (hcp_digital_sqrt**3)*hcp_digi_m3 + hcp_digi_int_C

    # HCP MAIL
    y_hcp_mail = hcp_mail_Sqrt*hcp_mail_m1 + (hcp_mail_Sqrt**2)*hcp_mail_m2 + hcp_mail_int_c

    # DTC DISPLAY 
    y_dtc_display = dtc_display_sqrt*dtc_disp_m1 + (dtc_display_sqrt**2)*dtc_disp_m2 + (dtc_display_sqrt**3)*dtc_disp_m3 + (dtc_display_sqrt**4)*dtc_disp_m4 + dtc_disp_intc

    # HCP DISPLAY 
    y_hcp_display = hcp_display_sqrt*hcp_disp_m1 + (hcp_display_sqrt**2)*hcp_disp_m2 + (hcp_display_sqrt**3)*hcp_disp_m3 + (hcp_display_sqrt**4)*hcp_disp_m4  + (hcp_display_sqrt**5)*hcp_disp_m5 + (hcp_display_sqrt**6)*hcp_disp_m6+ hcp_disp_intc


    # speaker program-person
    y_spk_person = spk_person_sqrt*sp_p_m1 + (spk_person_sqrt**2)*(sp_p_m2) + (spk_person_sqrt**3)*(sp_p_m3) + sp_p_intc

    # Speaker program Virtual 
    y_spk_virtual = spk_virtual_Sqrt*sp_v_m1 + (spk_virtual_Sqrt**2)*(sp_v_m2) + (spk_virtual_Sqrt**3)*(sp_v_m3) + sp_v_intc

    # GNA CALLS 
    y_gna_Calls = gna_call_sqrt*gna_call_m1 + (gna_call_sqrt**2)*(gna_call_m2)+ (gna_call_sqrt**3)*(gna_call_m3)+ (gna_call_sqrt**4)*(gna_call_m4) + gna_call_intc

    # DTC SOCIAL 
    y_dtc_social = (dtc_social_sqrt * dtc_social_m1) + (dtc_social_sqrt**2)*(dtc_social_m2) + (dtc_social_sqrt**3)*(dtc_social_m3) + (dtc_social_sqrt**4)*(dtc_social_m4) + (dtc_social_sqrt**5)*(dtc_social_m5) + dtc_social_intc



    model_2.Equations([fild_spend + dp_spend + hcp_spend + hcp_digital_spend + hcp_mail_spend + dtc_display_spend + hcp_display_spend + spk_person_spend + spk_virtual_spend + gna_Calls_spend +dtc_social_spend <= total_spend] )  # sum of spend on all channels == Total Spend

    model_2.Maximize(y_fild_calls + y_dp_search + y_hcp_search + y_hcp_digital + y_hcp_mail + y_dtc_display + y_hcp_display + y_spk_person + y_spk_virtual + y_gna_Calls + y_dtc_social  )    # #total revenue should be max 

    model_2.solve(disp=False) 
    return model_2 , constant_values

In [5]:
model_2 ,constants_vars = gekko_(total_budget=total_budget)

In [24]:
constants_vars

{'hcp_website_spend_': 483486,
 'dtc_website_spend_': 400000,
 'hcp_website_revenue_': 6800000,
 'dtc_website_revenue_': 9600000}

In [7]:
print(model_2._variables)
len([x[0] for x in model_2._variables])

[[52057876.591], [5490000.0], [262821.9], [1322086.5], [212278.0], [2230272.0], [1366905.1], [907275.6], [1134824.4], [990000.0], [2193100.8]]


11

In [19]:
model_2._constants


[i15, i16, i17]

In [16]:
hcp_website_revenue_.VALUE
dtc_website_revenue_.VALUE

9600000

In [8]:
model_2.options.OBJFCNVAL
# [x[0] for x in model_2._variables]

-155817302.94

In [9]:
hcp_website_spend_ = model_2.Const(483486)
dtc_website_spend_ = model_2.Const(400000)

In [9]:
model_2._variables

[[52057876.591],
 [5490000.0],
 [262821.9],
 [1322086.5],
 [212278.0],
 [2230272.0],
 [1366905.1],
 [907275.6],
 [1134824.4],
 [990000.0],
 [2193100.8],
 [128604.41133],
 [55018.913958]]

In [10]:
len([x[0] for x in model_2._variables])

11

## prediction : 

In [11]:
l = [ x[0] for x in model_2._variables]
Field_sqrt = l[0]**0.5
dp_sqrt = l[1]**0.5  
hcp_sqrt =l[2]**0.5 
hcp_digital_sqrt =  l[3]**0.5
hcp_mail_Sqrt = l[4]**0.5
dtc_display_sqrt = l[5]**0.5 
hcp_display_sqrt = l[6]**0.5       
spk_person_sqrt = l[7]**0.5
spk_virtual_Sqrt =  l[8]**0.5
gna_call_sqrt =l[9]**0.5 
dtc_social_sqrt  = l[10]**0.5 

In [12]:
y_fild_calls = (Field_sqrt*f_m1) + (Field_sqrt**2)*(f_m2) + (Field_sqrt**3)*(f_m3) + (Field_sqrt**4)*(f_m4) + (Field_sqrt**5)*(f_m5) + f_Intercept_ # Revenue Field calls

# DTC PAID SEARCH 
y_dp_search = dp_sqrt*(dtc_p_search_m1) + ((dp_sqrt**2)*(dtc_p_search_m2))  + dtc_p_search_int_c    # Revenue  DTC paid search

y_hcp_search = hcp_sqrt*(hcp_m1) + ( (hcp_sqrt**2)* hcp_m2) + hcp_int_c # Revenue HCP search 

# hcp digital 
y_hcp_digital  = hcp_digital_sqrt*(hcp_digi_m1) +  (hcp_digital_sqrt**2)*hcp_digi_m2 + (hcp_digital_sqrt**3)*hcp_digi_m3 + hcp_digi_int_C

# HCP MAIL
y_hcp_mail = hcp_mail_Sqrt*hcp_mail_m1 + (hcp_mail_Sqrt**2)*hcp_mail_m2 + hcp_mail_int_c

# DTC DISPLAY 
y_dtc_display = dtc_display_sqrt*dtc_disp_m1 + (dtc_display_sqrt**2)*dtc_disp_m2 + (dtc_display_sqrt**3)*dtc_disp_m3 + (dtc_display_sqrt**4)*dtc_disp_m4 + dtc_disp_intc


# 24250960.13

# HCP DISPLAY 
y_hcp_display = hcp_display_sqrt*hcp_disp_m1 + (hcp_display_sqrt**2)*hcp_disp_m2 + (hcp_display_sqrt**3)*hcp_disp_m3 + (hcp_display_sqrt**4)*hcp_disp_m4  + (hcp_display_sqrt**5)*hcp_disp_m5 + (hcp_display_sqrt**6)*hcp_disp_m6+ hcp_disp_intc


# # speaker program-person
y_spk_person = spk_person_sqrt*sp_p_m1 + (spk_person_sqrt**2)*(sp_p_m2) + (spk_person_sqrt**3)*(sp_p_m3) + sp_p_intc

# Speaker program Virtual 
y_spk_virtual = spk_virtual_Sqrt*sp_v_m1 + (spk_virtual_Sqrt**2)*(sp_v_m2) + (spk_virtual_Sqrt**3)*(sp_v_m3)+ sp_v_intc

# # GNA CALLS 
y_gna_Calls = gna_call_sqrt*gna_call_m1 + (gna_call_sqrt**2)*(gna_call_m2)+ (gna_call_sqrt**3)*(gna_call_m3)+ (gna_call_sqrt**4)*(gna_call_m4) + gna_call_intc

# DTC SOCIAL 
y_dtc_social = (dtc_social_sqrt * dtc_social_m1) + (dtc_social_sqrt**2)*(dtc_social_m2) + (dtc_social_sqrt**3)*(dtc_social_m3) + (dtc_social_sqrt**4)*(dtc_social_m4) + (dtc_social_sqrt**5)*(dtc_social_m5) + dtc_social_intc





In [13]:
print( 'PRedicted Revenue :  ' , y_fild_calls + y_dp_search + y_hcp_search + y_hcp_digital + y_hcp_mail + y_dtc_display + y_hcp_display + y_spk_person + y_spk_virtual+ y_gna_Calls+ y_dtc_social)
print('Optimizer Maximized Result : ' , model_2.options.OBJFCNVAL)

PRedicted Revenue :   155817302.94012907
Optimizer Maximized Result :  -155817302.94


In [149]:
print(' y_fild_calls : ' , y_fild_calls)
print(' y_dp_search : ' , y_dp_search)
print(' y_hcp_search : ' , y_hcp_search)
print(' y_hcp_digital : ' , y_hcp_digital)
print(' y_hcp_mail : ' , y_hcp_mail)
print(' y_dtc_display : ' , y_dtc_display)
print(' y_hcp_display : ' , y_hcp_display)
print('y_speaker_person' , y_spk_person)
print(' y_spk_virtual : ' , y_spk_virtual)
print(' y_gna_Calls : ' , y_gna_Calls)
print(' y_dtc_social : ' , y_dtc_social)


 y_fild_calls :  62386591.70530926
 y_dp_search :  2810106.9902107343
 y_hcp_search :  9554089.589205459
 y_hcp_digital :  407062.4944582753
 y_hcp_mail :  894875.7570143475
 y_dtc_display :  28639086.267569106
 y_hcp_display :  24549014.510930628
y_speaker_person 22505.168447354903
 y_spk_virtual :  248898.53421510113
 y_gna_Calls :  4622278.724343573
 y_dtc_social :  14831712.354535965


# **Excel report Generation**

In [159]:
from datetime import date


##### Result df
def generate_excel(Title , path='./'):
    '''
        AIM : This function will help to generate Excel report
        Inputs : 
            Title : Unique name of file 
            Path : path to store file ..
    
    '''
    today = date.today()
    result_dict = {    
                'Channels' :   ['Field calls' , 'DTC Paid Search' , 'HCP Paid Search' , 'HCP Digital' , 'HCP Email' , 'Dtc Display' , 'HCP Display' ,'Speaker Program Person' , 
                'Speaker Program Virtual' , 'GNA Calls' , 'DTC Social' ] ,         
                'mROI at Current Spend' : [] ,         
                'Current Spend' : [] ,  
                'Allocated Budget' : [x[0] for x in model_2._variables] , 

                'Predicted Revenue' : [y_fild_calls ,y_dp_search , y_hcp_search ,y_hcp_digital , y_hcp_mail ,  y_dtc_display ,y_hcp_display , y_spk_person ,  y_spk_virtual ,y_gna_Calls ,  y_dtc_social ] , 
                'Current Spend' : [] ,
                'Current Revenue' : [] 
                
            } 

    for i in result_dict['Channels'] : 
        df = pd.read_excel('./data/data_new.xlsx' , sheet_name=i, 
                        dtype={'% Change in Spend':str} )
        # print(np.where(df['% Change in Spend'] == '7.528699885739343e-16'))
        Current_data_ = df[df['% Change in Spend'] == '7.528699885739343e-16']
        result_dict['Current Spend'].append(Current_data_['Total Spend'].values[0])
        result_dict['Current Revenue'].append(Current_data_['Revenue'].values[0])
        result_dict['mROI at Current Spend'].append(Current_data_.iloc[:,-1].values[0]) 
    
    result_df = pd.DataFrame(result_dict)
    pd.options.display.float_format = "{:.2f}".format
    result_df = result_df.sort_values(by=['mROI at Current Spend'] , ascending=False)
    result_df['% change in revenue'] = ((result_df['Predicted Revenue'] - result_df['Current Revenue'] ) / result_df['Current Revenue'] )* 100
    result_df['% change in Spend'] = ((result_df['Allocated Budget'] - result_df['Current Spend'] ) / result_df['Current Spend'] )* 100

    additional_row = { 'Channels' : 'Total' , 'mROI at Current Spend' : np.nan , 'Current Spend' : result_df['Current Spend'].sum() , 
                    'Allocated Budget' : result_df['Allocated Budget'].sum(), 'Predicted Revenue' :result_df['Predicted Revenue' ].sum() , 'Current Revenue' :result_df['Current Revenue'].sum() ,
                     '% change in revenue' : np.nan, '% change in Spend' : np.nan }


    # arrnaging cols   
    result_df =result_df.append(additional_row , ignore_index=True)
    cols_sequence = ['Channels', 'mROI at Current Spend', 'Current Spend',
        'Allocated Budget', '% change in Spend', 'Current Revenue','Predicted Revenue', 
        '% change in revenue']
    result_df = result_df[cols_sequence]

    result_df.to_excel(f'{path}{Title}_Optimizer_Reports{today.strftime("%d_%m_%Y")}.xlsx')

    return result_df

b = generate_excel('_10' , path='../')
b.head()

,Channels,mROI at Current Spend,Current Spend,Allocated Budget,% change in Spend,Current Revenue,Predicted Revenue,% change in revenue
0,HCP Paid Search,13.98,195203.29,262821.90,34.64,8804973.37,9554089.59,8.51
1,Dtc Display,6.06,1440000.05,1924534.16,33.65,26100258.36,28639086.27,9.73
2,HCP Display,5.85,1036398.00,1224781.09,18.18,23737244.82,24549014.51,3.42
3,DTC Social,2.63,1536000.06,1794355.20,16.82,14194307.87,14831712.35,4.49
4,HCP Email,2.32,130745.01,173682.00,32.84,809427.92,894875.76,10.56


In [147]:
# b = a.copy()

# additional_row = { 'Channels' : 'Total' , 'mROI at Current Spend' : np.nan , 'Current Spend' : b['Current Spend'].sum() , 
#                     'Allocated Budget' : b['Allocated Budget'].sum(), 'Predicted Revenue' :b['Predicted Revenue' ].sum() , 'Current Revenue' : b['Current Revenue'].sum() ,
#                      '% change in revenue' : np.nan, '% change in Spend' : np.nan }
# b  =b.append(additional_row , ignore_index=True)
# b.head(15)

In [148]:
xl = pd.ExcelFile('./data/data_new.xlsx')
print(xl.sheet_names)

['Field calls', 'DTC Paid Search', 'GNA Calls', 'Speaker Program Virtual', 'Speaker Program Person', 'HCP Paid Search', 'Dtc Display', 'DTC Social', 'HCP Display', 'HCP Email', 'HCP Digital']


In [44]:
# df_ = pd.read_excel('./data/data_new.xlsx' , sheet_name='Field calls' , converters={'% Change in Spend':lambda value : '{}%'.format(value*100)})#
# df_.style.format({'% Change in Spend': "{:.0%}"})
# df_.head()
# df_['abc1'] = df_['% Change in Spend'].astype(str)

# df_['abc'] = df_['% Change in Spend'].str.rstrip("%").astype(float)/100
# df_[df_['abc1']== '7.528699885739343e-14%']
# df_.iloc[99:102]




In [ ]:
# float(7.528699885739343e-14)
# df_[df_['abc'] == 0.00]
# # df_[df_['% Change in Spend']== '7.528699885739343e-14%'] 
# df_[df_['% Change in Spend'] == 0.00]
# np.where(df_['% Change in Spend']==0.00)
# df_['% Change in Spend1'] =df_['% Change in Spend'].astype(str)


## **Dataframe**

In [60]:
df = pd.read_excel('./data/data_new.xlsx' , sheet_name='Field calls', 
                    dtype={'% Change in Spend':str} )
pd.options.display.float_format = "{:.2f}".format
df.iloc[99:102]

,% Change in Spend,Revenue,Total Spend,Field Calls_ROI,Field Calls mROI
99,-0.009999999999999247,66491795.74,52362198.04,1.27,0.66
100,7.528699885739343e-16,66837648.68,52891109.13,1.26,0.65
101,0.010000000000000753,67180721.52,53420020.22,1.26,0.65


In [13]:
df.columns

Index(['% Change in Spend', 'Revenue', 'Total Spend', 'Field Calls_ROI',
       'Field Calls mROI'],
      dtype='object')

In [61]:
result_dict = {     # add current budget into this , spend & revenue at current spend , mROI & store itin desc 
                # add col : % change in budget 
                # Actual Revenue : 
                # % change in Revenue 
                'Channels' :   ['Field calls' , 'DTC Paid Search' , 'HCP Paid Search' , 'HCP Digital' , 'HCP Email' , 'Dtc Display' , 'HCP Display' ,'Speaker Program Person' , 'Speaker Program Virtual' , 'GNA Calls' , 'DTC Social' ] ,                    
                'Allocated Budget' : [x[0] for x in model_2._variables] , 
                'Predicted Revenue' : [y_fild_calls ,y_dp_search , y_hcp_search ,y_hcp_digital , y_hcp_mail ,  y_dtc_display ,y_hcp_display , y_spk_person ,  y_spk_virtual ,y_gna_Calls ,  y_dtc_social ] , 
                'Current Spend' : [] ,
                'Current Revenue' : [] , 
                'mROI at Current Spend' : [] 
            } 

In [19]:
# ch = ['Field calls' , 'DTC Paid Search' , 'HCP Paid Search' , 'HCP Digital' , 'HCP Email' , 'Dtc Display' , 'HCP Display' ,'Speaker Program Person' , 'Speaker Program Virtual' , 'GNA Calls' , 'DTC Social' ] 

# j = 0 

# for i in ch : 
#     if j < 2 : 
#         df = pd.read_excel('./data/data_new.xlsx' , sheet_name=i, 
#                         dtype={'% Change in Spend':str} )
#         # print(np.where(df['% Change in Spend'] == '7.528699885739343e-16'))
#         a = df[df['% Change in Spend'] == '7.528699885739343e-16']
#         print(a['Total Spend'])
#         print(a['Revenue'])
        
#     else :
#         break 
#     j = j + 1 

    

## **results __ optimizer**

In [ ]:
print( 'PRedicted Revenue :  ' , y_fild_calls + y_dp_search + y_hcp_search + y_hcp_digital + y_hcp_mail + y_dtc_display + y_hcp_display + y_spk_person + y_spk_virtual+ y_gna_Calls+ y_dtc_social)
print('Optimizer Maximized Result : ' , model_2.options.OBJFCNVAL)

In [ ]:
print(' y_fild_calls : ' , y_fild_calls)
print(' y_dp_search : ' , y_dp_search)
print(' y_hcp_search : ' , y_hcp_search)
print(' y_hcp_digital : ' , y_hcp_digital)
print(' y_hcp_mail : ' , y_hcp_mail)
print(' y_dtc_display : ' , y_dtc_display)
print(' y_hcp_display : ' , y_hcp_display)
print('y_speaker_person' , y_spk_person)
print(' y_spk_virtual : ' , y_spk_virtual)
print(' y_gna_Calls : ' , y_gna_Calls)
print(' y_dtc_social : ' , y_dtc_social)

In [ ]:
# a = [ x[0] for x in model_2._variables]
# b= ['field call' ,'dtc paid search' , 'hcp paid search' , 'hcp_digital_spend' , 'hcp_mail_spend' , 'dtc_display_spend', 'hcp_display_spend' , 'spk_person_spend' , 'spk_virtual_spend' , 'gna_call_spend' , 'dtc_social_spend']
# mroi =[1.00,0.18,13.64,0.18,1.90,4.93,5.24,0.17,3.56,0.92,2.08]
# op = list(zip(b,mroi,a))
# op
# # less_than_10p

In [ ]:
# Field_sqrt = 45221898.0**0.5
# dp_sqrt = 7869000.0**0.5   #  2,68,299 

# hcp_sqrt =298660.5**0.5 
# hcp_digital_sqrt =  594939.0**0.5
# hcp_mail_Sqrt = 241224.0**0.5
# dtc_display_sqrt = 2764800.0**0.5 
# hcp_display_sqrt = 1694511.0**0.5       #  24250960.13
# spk_person_sqrt = 30239.376554**0.5
# spk_virtual_Sqrt =  18913.5**0.5

# gna_call_sqrt =924000.0**0.5 
# dtc_social_sqrt  = 2718720.00**0.5 

# cache

In [ ]:
# with few changes
js4 = '''{"channel_name":{"0":"Dtc_display","1":"DTC Social","2":"Dtc_paid_service"},"function_type":{"0":"log-log","1":"log-level","2":"log-level"},"r-square":{"0":0.9738176647,"1":0.97559292,"2":0.9755297402},"term_1 (a)":{"0":4.58815775,"1":0.33439696,"2":0.39308951},"term_2 (b)":{"0":4.90066613,"1":12.30889356,"2":8.64657667},"current_spend":{"0":100000,"1":100000,"2":100000},"min_spend":{"0":100000,"1":100000,"2":100000},
        "max_spend":{"0":310000,"1":490050,"2":240000},
        "total_budget":{"0":380000,"1":500000,"2":500000}
        ,"threshould":{"0":0.2,"1":0.4,"2":0.4}}'''

In [ ]:
from functools import lru_cache

@lru_cache(maxsize=10)
def optimizer(data) : 

    df = pd.read_json(data)
    channels_list = df.channel_name.values 
    Variables_dict = {} 
    equations_dict = {} 
    TOTAL_BUDGET = df['total_budget'].unique()[0]

    model=GEKKO() 

    for _ , row_ in df.iterrows():
        Variables_dict[row_['channel_name']] =  model.Var(value= row_['current_spend'], lb=row_['min_spend'],ub=row_['max_spend'] , name= row_['channel_name'] )
        
        def build_eq(row_ , knob=None) :
            equations_dict[row_['channel_name']] = model.Equation( Variables_dict[row_['channel_name']]  < row_['max_spend'] )  

        build_eq(row_ = row_ )  # constrains 
        model.Maximize( ( model._variables[_] * row_['term_1 (a)'] + row_['term_2 (b)']  ) ) ## Objective Function n

    model.Equation(sum(model._variables) == TOTAL_BUDGET) # constrains    
    model.solve(disp=False) 
  
    result_dictionary = dict(zip(channels_list , model._variables))
    result_dictionary['Objective'] = model.options.objfcnval
    print(result_dictionary)
    return result_dictionary

In [ ]:
optimizer(js4)